In [80]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

results = search.invoke("What is Nifty Index?")

print(results)

The NIFTY 50 index covers 13 sectors of the Indian economy and offers investment managers exposure to the Indian market in one portfolio. As of November 2025, NIFTY 50 gives a weightage of 36.84% to financial services including banking, 10.39% to oil and gas, 10.23% to information technology, 6.81% to automotive, and 6.49% to consumer goods. Nov 6, 2019 · The NIFTY share market index is a benchmark standard against which all equity markets in India are measured. Therefore, NSE conducts regular index maintenance to ensure that it remains stable and persists as the benchmark in the Indian stock market context. Dec 30, 2024 · Nifty is a stock market index used by the National Exchange of India (NSE) to track benchmark performance of the companies listed on the NSE—including NIFTY 50, NIFTY Next 50, and other key... Jul 30, 2025 · Nifty is a benchmark share market index that represents the Nifty 50 performance of the companies listed on the National Stock Exchange of India. Investors can u

In [81]:
print(search.name)
print(search.description)
print(search.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


## Custom Tools

In [82]:
from langchain_core.tools import tool

In [83]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [84]:
result = multiply.invoke({"a": 6, "b": 7})
print(result)

42


In [85]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


## Structured_tool and Pydantic

In [86]:
from langchain_core.tools import StructuredTool
from pydantic import BaseModel,Field   

In [87]:
class MultiplyInput(BaseModel):
    a :int = Field(required=True, description="The first number to multiply")
    b : int = Field(required=True, description="The second number to multiply")

/var/folders/yr/35cmrm35625bvzmvg6m3qrhw0000gn/T/ipykernel_96549/555859787.py:2: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  a :int = Field(required=True, description="The first number to multiply")
/var/folders/yr/35cmrm35625bvzmvg6m3qrhw0000gn/T/ipykernel_96549/555859787.py:3: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  b : int = Field(required=True, description="The second number to multiply")


In [88]:
def multiply_function(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [89]:
multiply_tool = StructuredTool.from_function(
    func=multiply_function,
    name="multiply",
    description="Multiply two numbers",
    args_schema=MultiplyInput
)

In [90]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to multiply', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to multiply', 'required': True, 'title': 'B', 'type': 'integer'}}


## Tool Calling

In [91]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [92]:
@tool
def multiply(a: int, b: int)-> int:
    """Multiply two numbers."""
    return a*b

In [93]:
multiply.name

'multiply'

In [94]:
multiply.description

'Multiply two numbers.'

In [95]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [96]:
llm = ChatOpenAI(model='gpt-5-nano')

In [97]:
llm_with_tool = llm.bind_tools([multiply])

In [98]:
llm_with_tool.invoke('Hi how are you?')

AIMessage(content='Hi! I’m here and ready to help. How can I assist you today? If you have a topic in mind—questions, writing, math, coding, brainstorming, or anything else—tell me and I’ll dive in.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 248, 'prompt_tokens': 130, 'total_tokens': 378, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CtaMjStxYNiBSTgqcK1MzmI53D5cm', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b7f18-6818-7b21-8726-3e9cdfe8c55d-0', usage_metadata={'input_tokens': 130, 'output_tokens': 248, 'total_tokens': 378, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning':

In [99]:
query = HumanMessage('can you multiply 3 with 100')

In [100]:
messages = [query]

In [101]:
messages

[HumanMessage(content='can you multiply 3 with 100', additional_kwargs={}, response_metadata={})]

In [102]:
result = llm_with_tool.invoke(messages)

In [103]:
result

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 221, 'prompt_tokens': 133, 'total_tokens': 354, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CtaMlZkeWsZQJQt0hM5a7O0AXc6V9', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b7f18-723b-7612-bc15-8416720cb4f1-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 100}, 'id': 'call_UslXPWDebddQ8Eq40hxFgzTM', 'type': 'tool_call'}], usage_metadata={'input_tokens': 133, 'output_tokens': 221, 'total_tokens': 354, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 192}})

In [104]:
result.tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 100},
  'id': 'call_UslXPWDebddQ8Eq40hxFgzTM',
  'type': 'tool_call'}]

In [105]:
messages.append(result)

In [106]:
messages

[HumanMessage(content='can you multiply 3 with 100', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 221, 'prompt_tokens': 133, 'total_tokens': 354, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CtaMlZkeWsZQJQt0hM5a7O0AXc6V9', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b7f18-723b-7612-bc15-8416720cb4f1-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 100}, 'id': 'call_UslXPWDebddQ8Eq40hxFgzTM', 'type': 'tool_call'}], usage_metadata={'input_tokens': 133, 'output_tokens': 221, 'total_tokens': 354, 'input_token_details': {'audio': 0, 'cache_read': 0}, 

In [107]:
tool_result = multiply.invoke(result.tool_calls[0])

In [108]:
tool_result

ToolMessage(content='300', name='multiply', tool_call_id='call_UslXPWDebddQ8Eq40hxFgzTM')

In [109]:
messages.append(tool_result)

In [110]:
messages

[HumanMessage(content='can you multiply 3 with 100', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 221, 'prompt_tokens': 133, 'total_tokens': 354, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CtaMlZkeWsZQJQt0hM5a7O0AXc6V9', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b7f18-723b-7612-bc15-8416720cb4f1-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 100}, 'id': 'call_UslXPWDebddQ8Eq40hxFgzTM', 'type': 'tool_call'}], usage_metadata={'input_tokens': 133, 'output_tokens': 221, 'total_tokens': 354, 'input_token_details': {'audio': 0, 'cache_read': 0}, 

In [111]:
llm_with_tool.invoke(messages).content

'3 multiplied by 100 equals 300.\n\nWould you like me to multiply anything else?'

In [112]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated
import requests

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """
    This function fetches the currency conversion factor between a given base currency and a target currency
    """
    url = f'https://v6.exchangerate-api.com/v6/32b1118bf6d47de44c9948e9/pair/{base_currency}/{target_currency}'

    response = requests.get(url)

    return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """
    given a currency conversion rate this function calculates the target currency value from a given base currency value
    """
    return base_currency_value * conversion_rate


In [113]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [114]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1767312001,
 'time_last_update_utc': 'Fri, 02 Jan 2026 00:00:01 +0000',
 'time_next_update_unix': 1767398401,
 'time_next_update_utc': 'Sat, 03 Jan 2026 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 90.034}

In [115]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

In [116]:
llm = ChatOpenAI()

In [117]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [118]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [119]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [120]:
ai_message = llm_with_tools.invoke(messages)

In [121]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': 'call_fzGToMa3icY43UIzYRMWesFT',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': 'call_Ztlp1jGhJtmK9wnmuAWoZd5x',
  'type': 'tool_call'}]

In [ ]:
llm1 =  ChatOpenAI()

In [123]:
llm1_with_tools = llm1.bind_tools([get_conversion_factor,convert])

In [124]:
messages1 = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [125]:
messages1

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [126]:
ai_message1 = llm1_with_tools.invoke(messages1)

In [127]:
messages1.append(ai_message1)

In [128]:
ai_message1.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': 'call_Ps5Bwjx9wwql6VVFbj9HPdza',
  'type': 'tool_call'}]